<a href="https://colab.research.google.com/github/mohansameer1983/ComputerVisionProjects/blob/main/Trash_Classification_CV_Neptune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Uncomment only for first run
!pip install neptune-client neptune-tensorflow-keras tensorflow

     |████████████████████████████████| 315 kB 4.3 MB/s 
     |████████████████████████████████| 829 kB 59.0 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 181 kB 66.4 MB/s 
     |████████████████████████████████| 132 kB 53.9 MB/s 
     |████████████████████████████████| 79 kB 10.2 MB/s 
     |████████████████████████████████| 8.6 MB 54.3 MB/s 
     |████████████████████████████████| 138 kB 73.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 127 kB 74.1 MB/s 
     |████████████████████████████████| 462 kB 71.2 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 130 kB 73.0 MB/s 
     |████████████████████████████████| 64 kB 2.9 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.15.1-py2.py3-none-any.whl size=564805 sha256=3b95c4b757a4393914307fe505992cf9f337a283e47b56f2856e25414df61b92
  Stored in directory: /ro

In [2]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime

import neptune.new as neptune
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

import cv2
from google.colab.patches import cv2_imshow      # To show images using cv2 module

import pathlib

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input, Dropout
from keras.preprocessing.image import ImageDataGenerator

import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.8.0
Hub version: 0.12.0
GPU is available


In [3]:
run = neptune.init(
    project="mohansameer/Trash-Classification-Deep-Learning",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5ZjI5MDY4Yi0zMWVlLTQ3MmMtYjE4NS1iNDRjNGM0OTg4ZDkifQ==",
) 

https://app.neptune.ai/mohansameer/Trash-Classification-Deep-Learning/e/TRAS-13
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


# Model Selection

In [4]:
  # Select base model  for transfer learning
  model_name = "resnet_v2_101" # @param [ 'resnet_v1_50', 'resnet_v1_101', 'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152',  'mobilenet_v2_100_224', 'mobilenet_v2_130_224', 'mobilenet_v2_140_224', 'mobilenet_v3_small_100_224', 'mobilenet_v3_small_075_224', 'mobilenet_v3_large_100_224', 'mobilenet_v3_large_075_224']

  model_handle_map = {
    "resnet_v1_50": "https://tfhub.dev/google/imagenet/resnet_v1_50/feature-vector/4",
    "resnet_v1_101": "https://tfhub.dev/google/imagenet/resnet_v1_101/feature-vector/4",
    "resnet_v1_152": "https://tfhub.dev/google/imagenet/resnet_v1_152/feature-vector/4",
    "resnet_v2_50": "https://tfhub.dev/google/imagenet/resnet_v2_50/feature-vector/4",
    "resnet_v2_101": "https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5",
    "resnet_v2_152": "https://tfhub.dev/google/imagenet/resnet_v2_152/feature-vector/4",
    "nasnet_large": "https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/4",
    "nasnet_mobile": "https://tfhub.dev/google/imagenet/nasnet_mobile/feature_vector/4",
    "pnasnet_large": "https://tfhub.dev/google/imagenet/pnasnet_large/feature_vector/4",
    "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
    "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/4",
    "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4",
    "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5",
    "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5",
    "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5",
    "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5",
  }

  PIXELS = 224#@param {type:"integer"}
  BATCH_SIZE = 16#@param {type:"integer"}

  model_handle = model_handle_map.get(model_name)

  print(f"Selected model: {model_name} : {model_handle}")

  IMAGE_SIZE = (PIXELS, PIXELS)
  print(f"Input size {IMAGE_SIZE}")


Selected model: resnet_v2_101 : https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5
Input size (224, 224)


In [5]:
#Log to NeptuneAI
run["model/transferlearning/model_name"] = model_name
run["model/transferlearning/model_url"] = model_handle
run["model/transferlearning/model_batch_size"] = BATCH_SIZE
run["model/parameters/image_size"] = IMAGE_SIZE

# Data Preparation

In [6]:
# Mount Google drive so dataset can be accessed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def load_data(save_dir="./"):
  data_dir = pathlib.Path('/content/drive/MyDrive/Colab Notebooks/data/trash') 

  datagen_kwargs = dict(rescale=1./255, validation_split=0.2)
  dataflow_kwargs = dict(target_size=(PIXELS, PIXELS), 
                        batch_size=BATCH_SIZE,
                        interpolation="nearest")

  valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      **datagen_kwargs)

  valid_generator = valid_datagen.flow_from_directory(
      data_dir, subset="validation", 
      shuffle=False, **dataflow_kwargs)
  
  train_datagen = ImageDataGenerator(
            horizontal_flip=True,
            **datagen_kwargs)

  train_generator = train_datagen.flow_from_directory(
          data_dir,
          class_mode='categorical',
          **dataflow_kwargs)
  return valid_generator, train_generator

VAL_generator, TRAIN_generator = load_data()

Found 288 images belonging to 28 classes.
Found 1500 images belonging to 28 classes.


In [8]:
class_names = TRAIN_generator.class_indices.items()
print(class_names)

dict_items([('Aluminium foil', 0), ('Battery', 1), ('Blister pack', 2), ('Bottle', 3), ('Bottle cap', 4), ('Broken glass', 5), ('Can', 6), ('Carton', 7), ('Cigarette', 8), ('Cup', 9), ('Food waste', 10), ('Glass jar', 11), ('Lid', 12), ('Other plastic', 13), ('Paper', 14), ('Paper bag', 15), ('Plastic bag & wrapper', 16), ('Plastic container', 17), ('Plastic glooves', 18), ('Plastic utensils', 19), ('Pop tab', 20), ('Rope & strings', 21), ('Scrap metal', 22), ('Shoe', 23), ('Squeezable tube', 24), ('Straw', 25), ('Styrofoam piece', 26), ('Unlabeled litter', 27)])


# Modelling

## Build and Train Model


In [10]:
def build_model(trainable=False, params={}):
   print("Building model with", model_handle)
   model = keras.Sequential([
        keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
        hub.KerasLayer(model_handle, trainable=trainable),
        keras.layers.Dense(params['dense_units_1'],activation=params['dense_layer_activation_1']),
        keras.layers.Dropout(params['dropout']),
        keras.layers.Dense(TRAIN_generator.num_classes,
                              kernel_regularizer=tf.keras.regularizers.l2(params['regularizer_rate']))])
   model.build((None,)+IMAGE_SIZE+(3,))
   model.summary()
   return model

params = {
    "dense_units_1": 128,
    "dense_layer_activation_1": "relu",
    "regularizer_rate":0.005,
    "dropout": 0.1,
    "learning_rate": 0.01,
    "n_epochs": 15,
}
run["model/parameters"] = params
model = build_model(params=params)

Building model with https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 2048)              42626560  
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 28)                3612      
                                                                 
Total params: 42,892,444
Trainable params: 265,884
Non-trainable params: 42,626,560
_________________________________________________________________


In [11]:
def compile_model(model):
  model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=params['learning_rate'], momentum=0.8), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])
  return model

model = compile_model(model)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
def train_model(model, num_epochs):

    steps_per_epoch = TRAIN_generator.samples // TRAIN_generator.batch_size
    validation_steps = VAL_generator.samples // VAL_generator.batch_size

    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

    history = model.fit(
      TRAIN_generator,
      epochs=num_epochs, steps_per_epoch=steps_per_epoch,
      validation_data = VAL_generator,
      validation_steps = validation_steps,
      callbacks=[NeptuneCallback(run=run)]).history
    
    return model, history

model, history = train_model(model, num_epochs=params['n_epochs'])

Epoch 1/15
93/93 [==============================] - 1127s 12s/step - loss: 3.1681 - accuracy: 0.2210 - val_loss: 2.7087 - val_accuracy: 0.3090
Epoch 2/15
93/93 [==============================] - 222s 2s/step - loss: 2.7753 - accuracy: 0.3147 - val_loss: 2.3981 - val_accuracy: 0.4028
Epoch 3/15
93/93 [==============================] - ETA: 0s - loss: 2.5622 - accuracy: 0.3571

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(history["loss"])
plt.plot(history["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(history["accuracy"])
plt.plot(history["val_accuracy"])

# Save Model

In [ ]:
saved_model_path = "/content/drive/MyDrive/Colab Notebooks/Models/saved_garbageclassification_model"
tf.saved_model.save(model, saved_model_path)